# Lab on Multi-Task Learning and Transfer Learning
## Introduction
In this lab, we will explore the concepts of Multi-Task Learning (MTL) and Transfer Learning. These are powerful techniques in machine learning used to leverage shared knowledge across tasks or domains. This lab is structured to provide a clear understanding of these concepts through explanations, coding exercises, and practical tasks using real-world datasets.

---


## Part 1: Multi-Task Learning
### Theory
Multi-Task Learning is a paradigm where a single model is trained on multiple related tasks simultaneously. It leverages shared representations to improve performance across all tasks. This approach is particularly beneficial when labeled data for individual tasks is limited or when tasks share underlying structures.

---

### Implementation
UTKFace dataset is a large-scale face dataset with long age span (range from 0 to 116 years old). The dataset consists of over 20,000 face images with annotations of age, gender, and ethnicity. The images cover large variation in pose, facial expression, illumination, occlusion, resolution, etc.

Link of the dataset: https://www.kaggle.com/datasets/jangedoo/utkface-new/data

#Downloading dataset from kaggle

In [ ]:
# Install Kaggle API
!pip install -q kaggle

# Upload kaggle.json file
from google.colab import files
files.upload()

# Move kaggle.json to the correct location and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the UTKFace dataset
!kaggle datasets download -d jangedoo/utkface-new

# Extract the dataset
import zipfile
with zipfile.ZipFile('utkface-new.zip', 'r') as zip_ref:
    zip_ref.extractall('UTKFace')


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
100% 331M/331M [00:15<00:00, 25.1MB/s]
100% 331M/331M [00:15<00:00, 22.5MB/s]


#Importing

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import cv2

#Constants and Dataset Paths

In [ ]:
# Constants
IMG_SIZE = 200  # The target size to which all images will be resized (200x200)
BATCH_SIZE = 32  # Number of images processed in each batch
EPOCHS = 20  # Number of training epochs
IMG_PATH = "/content/UTKFace/UTKFace"  # Path to the dataset on Kaggle


#Load Dataset (Filenames and Labels)

In [ ]:
# Load dataset (we're keeping just filenames and labels)
image_filenames = []  # List to store image filenames
ages = []  # List to store age labels
genders = []  # List to store gender labels

# Load image filenames and labels without loading images into memory yet
for img_name in os.listdir(IMG_PATH):  # Loop through all the image files in the dataset
    try:
        # Extract age and gender from filename (e.g., '25_0_0_0_0_0_0.jpg' => age=25, gender=0)
        age = int(img_name.split("_")[0])  # Age is the first value in the filename
        gender = int(img_name.split("_")[1])  # Gender is the second value in the filename

        # Append to lists
        image_filenames.append(img_name)
        ages.append(age)
        genders.append(gender)
    except Exception as e:  # Handle any errors (e.g., if the filename doesn't follow the expected pattern)
        print(f"Error processing {img_name}: {e}")

# Convert labels to numpy arrays
ages = np.array(ages, dtype=np.int64)  # Convert age list to numpy array
genders = np.array(genders, dtype=np.uint8)  # Convert gender list to numpy array


#Split Data into Training and Validation Sets

In [ ]:
# Split into training and validation sets
train_filenames, val_filenames, y_age_train, y_age_val, y_gender_train, y_gender_val = train_test_split(
    image_filenames, ages, genders, test_size=0.2, random_state=42  # Split 80% for training and 20% for validation
)

#Custom Data Generator for Loading Data in Batches

In [ ]:
# Custom data generator
def data_generator(filenames, labels_age, labels_gender, batch_size):
    while True:  # Infinite loop to keep yielding batches
        # Create batches
        for i in range(0, len(filenames), batch_size):  # Iterate over filenames in steps of batch_size
            batch_filenames = filenames[i:i+batch_size]  # Get the current batch of filenames
            batch_y_age = labels_age[i:i+batch_size]  # Get the corresponding age labels for the batch
            batch_y_gender = labels_gender[i:i+batch_size]  # Get the corresponding gender labels for the batch

            batch_images = []  # Initialize a list to store images for the current batch
            for filename in batch_filenames:
                img = cv2.imread(os.path.join(IMG_PATH, filename))  # Read the image from disk
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB format
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize the image to IMG_SIZE x IMG_SIZE
                batch_images.append(img)  # Add the processed image to the batch

            batch_images = np.array(batch_images, dtype=np.float32) / 255.0  # Normalize image pixel values to [0, 1]
            yield batch_images, {'age_output': batch_y_age, 'gender_output': batch_y_gender}  # Yield images and labels as a tuple


#Model Definition

In [ ]:
# Model definition
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))  # Input layer expects images of shape (200, 200, 3)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)  # 2D convolution with 32 filters and ReLU activation
x = MaxPooling2D((2, 2))(x)  # Max pooling with a 2x2 window
x = Conv2D(64, (3, 3), activation='relu')(x)  # 2D convolution with 64 filters
x = MaxPooling2D((2, 2))(x)  # Max pooling
x = Conv2D(128, (3, 3), activation='relu')(x)  # 2D convolution with 128 filters
x = MaxPooling2D((2, 2))(x)  # Max pooling
x = Flatten()(x)  # Flatten the output of the convolutions into a 1D vector
x = Dropout(0.5)(x)  # Dropout layer to reduce overfitting

In [ ]:
# Age prediction branch
age_output = Dense(1, name='age_output')(x)  # Dense layer for age prediction (regression output)

In [ ]:
# Gender prediction branch
gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)  # Dense layer for gender prediction (binary classification)

In [ ]:
# Combined model
model = Model(inputs=input_layer, outputs=[age_output, gender_output])  # The model has two outputs: age and gender

#Compile and Train the Model


In [ ]:
# Compile the model
model.compile(
    optimizer='adam',  # Adam optimizer for training
    loss={'age_output': 'mean_squared_error', 'gender_output': 'binary_crossentropy'},  # Different loss functions for each output
    metrics={'age_output': 'mae', 'gender_output': 'accuracy'}  # Metrics for evaluation
)

In [ ]:
# Train the model using the custom generator
history = model.fit(
    data_generator(train_filenames, y_age_train, y_gender_train, BATCH_SIZE),  # Training data generator
    validation_data=data_generator(val_filenames, y_age_val, y_gender_val, BATCH_SIZE),  # Validation data generator
    steps_per_epoch=len(train_filenames) // BATCH_SIZE,  # Number of steps per epoch
    validation_steps=len(val_filenames) // BATCH_SIZE,  # Number of validation steps
    epochs=EPOCHS  # Number of epochs
)


Epoch 1/20
592/592 ━━━━━━━━━━━━━━━━━━━━ 37s 49ms/step - age_output_loss: 459.4293 - age_output_mae: 16.2627 - gender_output_accuracy: 0.6525 - gender_output_loss: 0.6562 - loss: 460.0854 - val_age_output_loss: 248.8406 - val_age_output_mae: 11.6558 - val_gender_output_accuracy: 0.8157 - val_gender_output_loss: 0.4178 - val_loss: 249.2584
Epoch 2/20
592/592 ━━━━━━━━━━━━━━━━━━━━ 31s 45ms/step - age_output_loss: 230.1503 - age_output_mae: 11.6601 - gender_output_accuracy: 0.7884 - gender_output_loss: 0.5402 - loss: 230.8545 - val_age_output_loss: 184.4683 - val_age_output_mae: 10.6749 - val_gender_output_accuracy: 0.8435 - val_gender_output_loss: 0.4090 - val_loss: 185.3422
Epoch 3/20
592/592 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - age_output_loss: 180.3463 - age_output_mae: 10.2845 - gender_output_accuracy: 0.8274 - gender_output_loss: 0.4483 - loss: 180.9346 - val_age_output_loss: 162.5773 - val_age_output_mae: 10.0236 - val_gender_output_accuracy: 0.8601 - val_gender_output_loss: 0.3843 -

#Evaluate the Model

In [ ]:
# Evaluate the model
results = model.evaluate(data_generator(val_filenames, y_age_val, y_gender_val, BATCH_SIZE),
                         steps=len(val_filenames) // BATCH_SIZE)  # Evaluate on the validation set

print(f"Validation Age MAE: {results[1]}")  # Index 1 for 'age_output' MAE (mean absolute error)
print(f"Validation Gender Accuracy: {results[2]}")  # Index 2 for 'gender_output' accuracy


148/148 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - age_output_loss: 100.2454 - age_output_mae: 7.3410 - gender_output_accuracy: 0.8823 - gender_output_loss: 0.4218 - loss: 100.6672
Validation Age MAE: 99.45651245117188
Validation Gender Accuracy: 0.4134889543056488


## Part 2: Transfer Learning
### Theory
Transfer Learning involves using a pre-trained model on a related task and fine-tuning it for a specific new task. This is especially useful when labeled data for the new task is limited.

---

### Implementation
We will use a pre-trained MobileNet model

Link of Dataset: https://www.kaggle.com/datasets/nunenuh/pytorch-challange-flower-dataset/data

#Downloading dataset from kaggle

In [ ]:
# Install Kaggle API
!pip install -q kaggle

# Upload kaggle.json file (manually upload via the Colab interface)
from google.colab import files
files.upload()

# Move kaggle.json to the correct location and set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the UTKFace dataset
# Download the dataset using Kaggle API
!kaggle datasets download -d nunenuh/pytorch-challange-flower-dataset


# Unzip the dataset
!unzip /content/pytorch-challange-flower-dataset.zip -d /content/flower_dataset/


#Importing

In [ ]:
import numpy as np
import os
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf


#Constants and Dataset Paths

In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 102  # Number of flower species
DATASET_PATH = '/content/flower_dataset/dataset'  # Replace with the actual path

In [ ]:
# Load dataset paths
train_dir = os.path.join(DATASET_PATH, 'train')  # Path to the training set
valid_dir = os.path.join(DATASET_PATH, 'valid') # Path to the validation set

# Custom Data Generator for Loading Data in Batches

In [ ]:
# Custom data generator for loading data in batches
def data_generator(filenames, labels, batch_size, dataset_path, img_size=IMG_SIZE):
    while True:
        # Loop through the dataset in batches
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]  # Get the current batch of filenames
            batch_labels = labels[i:i+batch_size]  # Get the corresponding labels for the batch

            batch_images = []  # Initialize a list to hold the images for the current batch
            for filename in batch_filenames:
                img_path = os.path.join(dataset_path, filename)  # Get the full path of the image
                img = Image.open(img_path).resize((img_size, img_size))  # Open and resize the image
                img = np.array(img) / 255.0  # Normalize pixel values to the range [0, 1]
                batch_images.append(img)  # Append the image to the batch_images list

            batch_images = np.array(batch_images)  # Convert the list of images to a numpy array
            yield batch_images, batch_labels  # Yield the batch of images and corresponding labels

#Load Labels from Dataset

In [ ]:
# Load the labels
def load_labels(dataset_path):
    label_dict = {}  # Initialize an empty dictionary to map folder names to label indices
    for label_dir in os.listdir(dataset_path):  # Loop through the directories (which are labeled)
        label_dict[label_dir] = int(label_dir) - 1  # Map folder names to labels (0-indexed)
    return label_dict


#Get Filenames and Labels for Train and Validation Data

In [ ]:
# Get filenames and labels for train and validation datasets
def get_filenames_and_labels(dataset_path, label_dict):
    filenames = []
    labels = []
    for label_dir in os.listdir(dataset_path):  # Loop through each label directory
        label = label_dict[label_dir]  # Get the corresponding label for the folder
        label_path = os.path.join(dataset_path, label_dir)  # Get the path to the current label folder
        if os.path.isdir(label_path):  # Check if it is a directory
            for filename in os.listdir(label_path):  # Loop through each file in the label directory
                filenames.append(os.path.join(label_dir, filename))  # Append the file path to filenames
                labels.append(label)  # Append the corresponding label
    return filenames, np.array(labels)  # Return the filenames and their corresponding labels


#Load Data and Set Up the Model

In [ ]:
# Load the labels and filenames
label_dict = load_labels(train_dir)  # Load labels for the training dataset
train_filenames, y_train = get_filenames_and_labels(train_dir, label_dict)  # Get filenames and labels for training
val_filenames, y_val = get_filenames_and_labels(valid_dir, label_dict)  # Get filenames and labels for validation


#Load Pre-trained VGG16 Model and Freeze Base Layers

In [ ]:
# Load pre-trained VGG16 model + higher level layers (without the top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False  # Set all layers in the base model to non-trainable

#Build and Compile the Custom Model

In [ ]:
# Build the model
x = Flatten()(base_model.output)  # Flatten the output of the base model
x = Dropout(0.5)(x)  # Add dropout to reduce overfitting
x = Dense(102, activation='softmax')(x)  # Add a dense layer with 102 output units (one per class) with softmax activation


In [ ]:
# Combine base model with the custom layers
model = Model(inputs=base_model.input, outputs=x)

#Complie Model

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train Model

In [ ]:
# Train the model using the custom generator
steps_per_epoch = len(train_filenames) // BATCH_SIZE  # Number of steps per epoch (how many batches per epoch)
validation_steps = len(val_filenames) // BATCH_SIZE  # Number of validation steps (how many validation batches)

In [ ]:
history = model.fit(
    data_generator(train_filenames, y_train, BATCH_SIZE, train_dir),
    validation_data=data_generator(val_filenames, y_val, BATCH_SIZE, valid_dir),
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=EPOCHS
)

Epoch 1/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 78s 377ms/step - accuracy: 0.0810 - loss: 19.0212 - val_accuracy: 0.0250 - val_loss: 27.9913
Epoch 2/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 91s 392ms/step - accuracy: 0.0024 - loss: 26.9267 - val_accuracy: 0.0471 - val_loss: 31.5160
Epoch 3/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 344ms/step - accuracy: 0.0400 - loss: 24.8290 - val_accuracy: 0.0611 - val_loss: 29.0145
Epoch 4/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 343ms/step - accuracy: 0.0880 - loss: 18.5215 - val_accuracy: 0.1896 - val_loss: 15.6842
Epoch 5/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 67s 330ms/step - accuracy: 0.2108 - loss: 12.2000 - val_accuracy: 0.2176 - val_loss: 15.4504
Epoch 6/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 70s 345ms/step - accuracy: 0.3602 - loss: 8.9980 - val_accuracy: 0.2774 - val_loss: 14.0036
Epoch 7/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 71s 349ms/step - accuracy: 0.4432 - loss: 7.6736 - val_accuracy: 0.3588 - val_loss: 10.8671
Epoch 8/10
204/204 ━━━━━━━━━━━━━━━━━━━━ 69s 342ms/step - accuracy: 0.52

#Evalute the Model

In [ ]:
# Evaluate the model
results = model.evaluate(data_generator(val_filenames, y_val, BATCH_SIZE, valid_dir), steps=validation_steps)
print(f"Validation Loss: {results[0]}")
print(f"Validation Accuracy: {results[1]}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step - accuracy: 0.2576 - loss: 11.4786
Validation Loss: 9.087573051452637
Validation Accuracy: 0.4112499952316284



#Assignment: Multi-Task Learning for Fashion Product Image Classification
##Overview:
In this assignment, you will use Multi-Task Learning (MTL) to classify fashion product images into three categories using the Fashion Product Images (Small) dataset:

* Article Category Classification (e.g., T-shirt, Jeans)
* Base Color Classification (e.g., Red, Blue)
* Target Season Classification (e.g., Summer, Winter)

**You will build a single model that predicts all three tasks simultaneously by sharing the same feature extraction layers and using separate output branches for each task.**

Link to dataset: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small

In [20]:
# Import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
# Download dataset
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): MIT
 98% 553M/565M [00:03<00:00, 144MB/s]
100% 565M/565M [00:03<00:00, 148MB/s]


In [3]:
# Unzip the dataset
with zipfile.ZipFile("fashion-product-images-small.zip", 'r') as zip_ref:
    zip_ref.extractall("fashion_dataset")

In [28]:
# Paths to the dataset directories
data_dir = "fashion_dataset/myntradataset/images"
metadata_path = "fashion_dataset/myntradataset/styles.csv"

In [29]:
# Load metadata
df= pd.read_csv(metadata_path, on_bad_lines='skip')
df.dropna(subset=['baseColour', 'season', 'articleType'], inplace=True)

In [30]:
# Filter metadata to include only rows with available images
df['image_path'] = df['id'].astype(str) + ".jpg"
df = df[df['image_path'].apply(lambda x: os.path.isfile(os.path.join(data_dir, x)))]

In [31]:
# Encode labels
article_encoder = LabelEncoder()
color_encoder = LabelEncoder()
season_encoder = LabelEncoder()

df['articleType'] = article_encoder.fit_transform(df['articleType'])
df['baseColour'] = color_encoder.fit_transform(df['baseColour'])
df['season'] = season_encoder.fit_transform(df['season'])


In [32]:
# Split dataset
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Define paths for train and validation image sets
train_image_paths = train_df['image_path'].apply(lambda x: os.path.join(data_dir, x)).tolist()
valid_image_paths = valid_df['image_path'].apply(lambda x: os.path.join(data_dir, x)).tolist()

train_labels = train_df[['articleType', 'baseColour', 'season']].values
valid_labels = valid_df[['articleType', 'baseColour', 'season']].values

In [33]:
# Define a custom data generator
class MultiTaskDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.num_samples = len(image_paths)

    def __len__(self):
        return int(np.ceil(self.num_samples / self.batch_size))

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        article_labels = []
        color_labels = []
        season_labels = []

        for i, path in enumerate(batch_image_paths):
            img = load_img(path, target_size=(128, 128))
            img = img_to_array(img) / 255.0
            images.append(img)

            article_labels.append(batch_labels[i][0])
            color_labels.append(batch_labels[i][1])
            season_labels.append(batch_labels[i][2])

        return (
            np.array(images),
            {
                'article_output': np.array(article_labels),
                'color_output': np.array(color_labels),
                'season_output': np.array(season_labels),
            }
        )

In [34]:
# Create train and validation generators
batch_size = 32
train_generator = MultiTaskDataGenerator(train_image_paths, train_labels, batch_size)
valid_generator = MultiTaskDataGenerator(valid_image_paths, valid_labels, batch_size)

In [35]:
# Build the model
input_layer = Input(shape=(128, 128, 3))

In [36]:
# Shared feature extractor
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)

In [37]:
# Output branches
article_output = layers.Dense(len(article_encoder.classes_), activation='softmax', name='article_output')(x)
color_output = layers.Dense(len(color_encoder.classes_), activation='softmax', name='color_output')(x)
season_output = layers.Dense(len(season_encoder.classes_), activation='softmax', name='season_output')(x)

In [38]:
# Create the model
model = models.Model(inputs=input_layer, outputs=[article_output, color_output, season_output])

In [41]:
# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'article_output': 'sparse_categorical_crossentropy',
        'color_output': 'sparse_categorical_crossentropy',
        'season_output': 'sparse_categorical_crossentropy',
    },
    metrics={
        'article_output': 'accuracy',
        'color_output': 'accuracy',
        'season_output': 'accuracy',
    }  # Specify metrics for each output by name
)

In [42]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(valid_generator)
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1110/1110 ━━━━━━━━━━━━━━━━━━━━ 60s 47ms/step - article_output_accuracy: 0.5779 - article_output_loss: 1.8451 - color_output_accuracy: 0.4545 - color_output_loss: 2.0376 - loss: 4.8798 - season_output_accuracy: 0.5883 - season_output_loss: 0.9973 - val_article_output_accuracy: 0.7806 - val_article_output_loss: 0.8175 - val_color_output_accuracy: 0.5916 - val_color_output_loss: 1.4532 - val_loss: 3.0727 - val_season_output_accuracy: 0.6631 - val_season_output_loss: 0.8005
Epoch 2/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - article_output_accuracy: 0.0000e+00 - article_output_loss: 0.0000e+00 - color_output_accuracy: 0.0000e+00 - color_output_loss: 0.0000e+00 - loss: 0.0000e+00 - season_output_accuracy: 0.0000e+00 - season_output_loss: 0.0000e+00
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1110/1110 ━━━━━━━━━━━━━━━━━━━━ 58s 30ms/step - article_output_accuracy: 0.8320 - article_output_loss: 0.5735 - color_output_accuracy: 0.6294 - color_output_loss: 1.2340 - loss: 2.5350 - season_output_accuracy: 0.6972 - season_output_loss: 0.7278 - val_article_output_accuracy: 0.8033 - val_article_output_loss: 0.7081 - val_color_output_accuracy: 0.5946 - val_color_output_loss: 1.4509 - val_loss: 2.8886 - val_season_output_accuracy: 0.7052 - val_season_output_loss: 0.7286
Epoch 4/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step - article_output_accuracy: 0.0000e+00 - article_output_loss: 0.0000e+00 - color_output_accuracy: 0.0000e+00 - color_output_loss: 0.0000e+00 - loss: 0.0000e+00 - season_output_accuracy: 0.0000e+00 - season_output_loss: 0.0000e+00
Epoch 5/10
1110/1110 ━━━━━━━━━━━━━━━━━━━━ 34s 30ms/step - article_output_accuracy: 0.8994 - article_output_loss: 0.3195 - color_output_accuracy: 0.7041 - color_output_loss: 0.9573 - loss: 1.9207 - season_output_accuracy: 0.7424 - season_out

In [43]:
# Save the model
model.save('multi_task_fashion_model.h5')

In [44]:
# Evaluate the model
results = model.evaluate(valid_generator)
print(f"Test Loss and Accuracy: {results}")

278/278 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - article_output_accuracy: 0.8162 - article_output_loss: 0.8354 - color_output_accuracy: 0.6017 - color_output_loss: 1.7153 - loss: 3.3269 - season_output_accuracy: 0.7094 - season_output_loss: 0.7762
Test Loss and Accuracy: [3.3826420307159424, 0.8399555683135986, 1.7574396133422852, 0.7840583324432373, 0.821110725402832, 0.596372663974762, 0.7100371718406677]
